In [4]:
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write as wav_write
from pynput import keyboard
import numpy as np

In [12]:
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write as wav_write
from pynput import keyboard

# Parameters for sound recording
sample_rate = 44100  # Hz
duration = 3  # seconds
audio_data = np.zeros((duration * sample_rate, 1), dtype=np.int16)

# File paths
audio_file = 'keyboard_sound.wav'
log_file = 'key_log.csv'

# Initialize the keystroke log file
with open(log_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key', 'Action', 'Timestamp'])

# Recording function for audio
def record_audio():
    global audio_data
    print("Recording audio...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()
    print("Audio recording finished")

    # Save the recorded audio to a WAV file
    wav_write(audio_file, sample_rate, audio_data)
    print(f"Audio saved to {audio_file}")

# Keystroke listener function
def on_press(key):
    timestamp = time.time()
    try:
        key_str = key.char  # Normal keys
    except AttributeError:
        key_str = str(key)  # Special keys like shift, ctrl, etc.

    # Log the key press with timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Pressed', timestamp])

    print(f"Key {key_str} Pressed at {timestamp}")

def on_release(key):
    timestamp = time.time()
    try:
        key_str = key.char
    except AttributeError:
        key_str = str(key)

    # Log the key release with timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Released', timestamp])

    print(f"Key {key_str} Released at {timestamp}")

    # Stop listener if 'Esc' key is pressed
    if key == keyboard.Key.esc:
        return False

# Start recording audio in a separate thread
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# Start the keyboard listener in the main thread to avoid blocking
def start_keyboard_listener():
    with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

# Run the keyboard listener in a separate thread
keyboard_thread = threading.Thread(target=start_keyboard_listener)
keyboard_thread.start()

# Wait for both threads to finish
audio_thread.join()
keyboard_thread.join()


Recording audio...
Audio recording finished
Audio saved to keyboard_sound.wav


In [3]:
# now extract key strokes from the keystroke timestamp

with open("key_log.csv","r") as audio_data
    reader = csv.reader(audio_data)
    keystroke_times = []
    for row in reader:
        # create a stack that gets all the keys pressed. If there are more than one pressed for example shift + a, then we will have a stack of keys
